# FARM


> Framework for Adapting Representation Models

## Core Features

- **Easy fine-tuning of language models** to your task and domain language
- **Speed**: AMP optimizers (~35% faster) and parallel preprocessing (16 CPU cores => ~16x faster)
- **Modular design** of language models and prediction heads
- Switch between heads or combine them for **multitask learning**
- **Full Compatibility** with HuggingFace Transformers' models and model hub
- **Smooth upgrading** to newer language models
- Integration of **custom datasets** via Processor class
- Powerful **experiment tracking** & execution
- **Checkpointing & Caching** to resume training and reduce costs with spot instances
- Simple **deployment** and **visualization** to showcase your model

In [4]:
import torch
import argparse
from pathlib import Path
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor, SquadProcessor
from farm.data_handler.data_silo import DataSilo
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import TextClassificationHead, QuestionAnsweringHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.optimization import initialize_optimizer
from farm.train import Trainer
from farm.utils import MLFlowLogger

In [3]:
parser = argparse.ArgumentParser(description="run farm")
parser.add_argument("--tracking_uri", type=str, default="https://public-mlflow.deepset.ai/",
    help="MLFlow - tracking uri ")
parser.add_argument("--experiment_name", type=str, default="FARM_tutorial",
    help="MLFlow - experiment name")
parser.add_argument("--run_name", type=str, default="NSMC",
    help="MLFlow - run name")
parser.add_argument("--pretrained_model_name_or_path", type=str, default="beomi/kcbert-base",
    help="Tokenizer, LanguageModel - pretrained model name")

parser.add_argument("--train_filename", type=str, default="train.tsv",
    help="Processor - train file name")
parser.add_argument("--test_filename", type=str, default="test.tsv",
    help="Processor - test file name")
parser.add_argument("--max_seq_len", type=int, default=150,
    help="Processor - max sequence lenght of tokens")
parser.add_argument("--data_dir",  type=str, default="./nsmc/",
    help="Processor - data directory")
parser.add_argument("--label_list", nargs="*", default=["bad", "good"],
    help="Processor - label list with string")
parser.add_argument("--metric",  type=str, default="acc",
    help="Processor - acc or f1_macro")
parser.add_argument("--label_column_name",  type=str, default="label",
    help="Processor - label column name")
parser.add_argument("--text_column_name",  type=str, default="document",
    help="Processor - text column name")
parser.add_argument("--ckpt_path",  type=str, default="./ckpt",
    help="Processor - checkpoint to save processor")
parser.add_argument("--batch_size", type=int, default=256,
    help="DataSilo - train batch size")
parser.add_argument("--eval_batch_size", type=int, default=256,
    help="DataSilo - eval batch size")
parser.add_argument("--embeds_dropout_prob", type=float, default=0.1,
    help="AdaptiveModel - The probability that a value in the embeddings returned by the language model will be zeroed.")
parser.add_argument("--learning_rate", type=float, default=2e-5,
    help="initialize_optimizer - learning rate")
parser.add_argument("--n_epochs", type=int, default=1,
    help="initialize_optimizer - number of epochs")
parser.add_argument("--n_gpu", type=int, default=4,
    help="Trainer - number of gpus")
parser.add_argument("--checkpoint_root_dir", type=str, default="./ckpt",
    help="Trainer - checkpoint root directory")
parser.add_argument("--checkpoints_to_keep", type=int, default=3,
    help="Trainer - number of checkpoint to keep")
parser.add_argument("--checkpoint_every", type=int, default=200,
    help="Trainer - checkpoint every")
parser.add_argument("--evaluate_every", type=int, default=200,
    help="Trainer - evaluate steps")
args = parser.parse_known_args()

# Total Process

```
Processor > DataSilo > Build Model(LanguageModel, TextClassificationHead, AdaptiveModel) > Build Optimizer & Trainer(initialize_optimizer, Trainer) > Train!
```

In [ ]:
tokenizer = Tokenizer.load(
    pretrained_model_name_or_path=args.pretrained_model_name_or_path,
    do_lower_case=False
)

# Comparing to transformers with pytorch-lightning